# Open In Colab

### Open this notebook in google colab

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1Bg8MQnfRZpc2Wy52XEpqaBtsopmA0Hoc?authuser=1#scrollTo=gGnaQzjyhxhu)


# Install & Import Module

In [ ]:
!pip install pytesseract
! pip install easyocr
!sudo apt install tesseract-ocr


  Created wheel for pytesseract: filename=pytesseract-0.3.6-py2.py3-none-any.whl size=13629 sha256=2b0706a299deae8a4c2225045589626c9777dd3b142297b852c603fd8caaf894
  Stored in directory: /root/.cache/pip/wheels/ee/71/72/b98430261d849ae631e283dfc7ccb456a3fb2ed2205714b63f
Successfully built pytesseract
     |████████████████████████████████| 48.9MB 59kB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 12 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr

In [ ]:
# ! sudo apt-get update
# !sudo apt-get install libleptonica-dev 
# ! sudo apt-get install tesseract-ocr tesseract-ocr-dev
# ! sudo apt-get install libtesseract-dev

In [ ]:
import matplotlib.pyplot as plt
import cv2, os
import easyocr
from pylab import rcParams
from IPython.display import Image
import nltk
import matplotlib.pyplot as plt 
import pytesseract


import cv2
import numpy as np
import dlib
from PIL import ImageColor
from datetime import datetime
from imutils import paths
import glob
import re
from imutils.object_detection import non_max_suppression
import numpy as np
import argparse
import time
import cv2


%matplotlib inline
rcParams['figure.figsize'] = 8, 16
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
en_reader = easyocr.Reader(['en'])

# Download Required File

In [ ]:
! wget https://raw.githubusercontent.com/AKSHAYUBHAT/TensorFace/master/openface/models/dlib/shape_predictor_68_face_landmarks.dat
! wget https://raw.githubusercontent.com/oyyd/frozen_east_text_detection.pb/master/frozen_east_text_detection.pb

--2020-11-13 14:09:13--  https://raw.githubusercontent.com/AKSHAYUBHAT/TensorFace/master/openface/models/dlib/shape_predictor_68_face_landmarks.dat
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 99693937 (95M) [application/octet-stream]
Saving to: ‘shape_predictor_68_face_landmarks.dat’

shape_predictor_68_ 100%[===================>]  95.08M   325MB/s    in 0.3s    

2020-11-13 14:09:15 (325 MB/s) - ‘shape_predictor_68_face_landmarks.dat’ saved [99693937/99693937]

--2020-11-13 14:09:15--  https://raw.githubusercontent.com/oyyd/frozen_east_text_detection.pb/master/frozen_east_text_detection.pb
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubu

# Making Folder

In [ ]:
if os.path.exists('/content/manual-cropedd'):
    pass
else:
    os.mkdir('/content/manual-cropedd')


if os.path.exists('/content/east-cropedd'):
    pass
else:
    os.mkdir('/content/east-cropedd')

# Dataset Making - Implemenation

In [ ]:
def EAST_DATASET():

    args = {
    # "image" : "/home/alpha/Desktop/opencv-text-detection/images/car_wash.png",
    "image" : '/content/NID_cropped.png',
    "east" : '/content/frozen_east_text_detection.pb',
    "min_confidence" : 0.5,
    "width" : 320,
    "height" : 320

    }

    # load the input image and grab the image dimensions
    image = cv2.imread(args["image"])
    orig = image.copy()
    (H, W) = image.shape[:2]

    # set the new width and height and then determine the ratio in change
    # for both the width and height
    (newW, newH) = (args["width"], args["height"])
    rW = W / float(newW)
    rH = H / float(newH)

    # resize the image and grab the new image dimensions
    image = cv2.resize(image, (newW, newH))
    (H, W) = image.shape[:2]

    # define the two output layer names for the EAST detector model that
    # we are interested -- the first is the output probabilities and the
    # second can be used to derive the bounding box coordinates of text
    layerNames = [
        "feature_fusion/Conv_7/Sigmoid",
        "feature_fusion/concat_3"]

    # load the pre-trained EAST text detector
    print("[INFO] loading EAST text detector...")
    net = cv2.dnn.readNet(args["east"])

    # construct a blob from the image and then perform a forward pass of
    # the model to obtain the two output layer sets
    blob = cv2.dnn.blobFromImage(image, 1.0, (W, H),
        (123.68, 116.78, 103.94), swapRB=True, crop=False)
    start = time.time()
    net.setInput(blob)
    (scores, geometry) = net.forward(layerNames)
    end = time.time()

    # show timing information on text prediction
    print("[INFO] text detection took {:.6f} seconds".format(end - start))

    # grab the number of rows and columns from the scores volume, then
    # initialize our set of bounding box rectangles and corresponding
    # confidence scores
    (numRows, numCols) = scores.shape[2:4]
    rects = []
    confidences = []

    # loop over the number of rows
    for y in range(0, numRows):
        # extract the scores (probabilities), followed by the geometrical
        # data used to derive potential bounding box coordinates that
        # surround text
        scoresData = scores[0, 0, y]
        xData0 = geometry[0, 0, y]
        xData1 = geometry[0, 1, y]
        xData2 = geometry[0, 2, y]
        xData3 = geometry[0, 3, y]
        anglesData = geometry[0, 4, y]

        # loop over the number of columns
        for x in range(0, numCols):
            # if our score does not have sufficient probability, ignore it
            if scoresData[x] < args["min_confidence"]:
                continue

            # compute the offset factor as our resulting feature maps will
            # be 4x smaller than the input image
            (offsetX, offsetY) = (x * 4.0, y * 4.0)

            # extract the rotation angle for the prediction and then
            # compute the sin and cosine
            angle = anglesData[x]
            cos = np.cos(angle)
            sin = np.sin(angle)

            # use the geometry volume to derive the width and height of
            # the bounding box
            h = xData0[x] + xData2[x]
            w = xData1[x] + xData3[x]

            # compute both the starting and ending (x, y)-coordinates for
            # the text prediction bounding box
            endX = int(offsetX + (cos * xData1[x]) + (sin * xData2[x]))
            endY = int(offsetY - (sin * xData1[x]) + (cos * xData2[x]))
            startX = int(endX - w)
            startY = int(endY - h)

            # add the bounding box coordinates and probability score to
            # our respective lists
            rects.append((startX, startY, endX, endY))
            confidences.append(scoresData[x])

    # apply non-maxima suppression to suppress weak, overlapping bounding
    # boxes
    boxes = non_max_suppression(np.array(rects), probs=confidences)

    # loop over the bounding boxes
    img_no = 1
    for (startX, startY, endX, endY) in boxes:
        # scale the bounding box coordinates based on the respective
        # ratios
        startX = int(startX * rW)
        startY = int(startY * rH)
        endX = int(endX * rW)
        endY = int(endY * rH)

        # draw the bounding box on the image
        #cv2.rectangle(orig, (startX-5, startY-5), (endX+10, endY+10), (0, 255, 0), 2)
        try:
            img_name = "/content/east-cropedd/"+str(img_no)+".png"
            roi = orig[startY-10:endY+15, startX-15:endX+145]
            cv2.imwrite(img_name, roi)

            img_no = img_no + 1
        except:
            try:
                img_name = "/content/east-cropedd/"+str(img_no)+".png"
                roi = orig[startY-10:endY+10, startX-5:endX+130]
                cv2.imwrite(img_name, roi)

                img_no = img_no + 1
            except:
                img_name = "/content/east-cropedd/"+str(img_no)+".png"
                roi = orig[startY-5:endY+10, startX-5:endX+130]
                cv2.imwrite(img_name, roi)

                img_no = img_no + 1

In [ ]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("/content/shape_predictor_68_face_landmarks.dat")

def make_dataset(img_path, isDelete=True):

    if isDelete:
        files = glob.glob('/content/manual-cropedd/*')
        for f in files:
            os.remove(f)
        files = glob.glob('/content/east-cropedd/*')
        for f in files:
            os.remove(f)

    # extrction information part from croped image based face
    filename = img_path
    img = cv2.imread(cv2.samples.findFile(filename))
    img = cv2.resize(img, (640, 480))

    imgOriginal = img.copy()
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    faces = detector(imgOriginal)
    for face in faces:
        x1,y1 = face.left(),face.top()
        x2,y2 = face.right(),face.bottom()


        cropped = imgOriginal[y1-25:y2+150 , x2+10:x2 + 320]

        cv2.imwrite("NID_cropped.png", cropped)
    
        # cropped each infromation and saved it in a directory
        img = cropped
        x_val, y_val, c = img.shape

        x , y = 0, 0
        for i in range(y_val):
            x = 0
            y = y + 10
            w, h = y_val, 60


            if y+10 > y_val:
                break

            try:
                crop_img = img[y:y+h, x:x+w]
                img_name = "/content/manual-cropedd/"+str(i)+".jpg"
                cv2.imwrite(img_name, crop_img)
            except:
                pass

    # Dataset for EAST model
    EAST_DATASET()
    # End Dataset function


# Easy-OCR Implementation

### Requeired function - Easy OCR

In [ ]:

MONTH_NAME = ['Jan', 'Feb', 'Apr' 'Mar', 'May', 'Jun', 'Jul', 'Aug', 'Sept', 'Oct', 'Nov', 'Dec']



def EASY_month_matching(sent):
    sent =  sent.strip()
    word_token = nltk.tokenize.word_tokenize(sent)
    for i, word in enumerate(word_token):
        if word in MONTH_NAME:
            date_c = word_token[i-1]
            s_d = re.search(r"\d+(\.\d+)?", date_c)
            date = s_d.group(0)

            month = word_token[i]


            year = word_token[i+1]
            

            b_date = date + " " + month + " " + year
            # print(b_date)
            return b_date

def EASY_NID_matching(sent):
    sent =  sent.strip()
    word_token = nltk.tokenize.word_tokenize(sent)
    for i, word in enumerate(word_token):
        try:
            if word_token[i].isdigit() and word_token[i+1].isdigit() and word_token[i+2].isdigit():
                if len(word_token[i]) >= 3 and len(word_token[i+1]) >= 3 and len(word_token[i+1]) >= 3 :
                    NID_no = word_token[i] +" " + word_token[i+1] +" " +  word_token[i+2]
                    return NID_no
        except:
            pass


### Info extraction functon  - Easy OCR

In [ ]:
def Extract_NID_INFO_EASY_OCR(img_path):
    
    img = plt.imread(img_path)
    plt.imshow(img)

    name = 'Not Found'
    birthday = 'Not Found'
    NID_number = 'Not Found'
    info_dict = dict()
    en_output = en_reader.readtext(img_path)
    for out in en_output:
        sent = out[1]
        res = [char for char in sent if char.isupper()]
        if len(res) > 6:
            name = sent
        
        b_date = EASY_month_matching(sent)
        NID_no = EASY_NID_matching(sent)
        if b_date != None:
            birthday = b_date
        if NID_no != None:
            NID_number = NID_no
            


    info_dict["Name"] = name
    info_dict["Date of Birth"] = birthday
    info_dict["NID_no"] = NID_number

    return info_dict

# PYTESSEROCR Implementation

In [ ]:
def PYTESS_OCR_month_matching(sent):
    sent =  sent.strip()
    word_token = nltk.tokenize.word_tokenize(sent)
    for i, word in enumerate(word_token):
        if word in MONTH_NAME:
            date = word_token[i-1]

            month = word_token[i]


            year = word_token[i+1]
            

            b_date = date + " " + month + " " + year
            # print(b_date)
            return b_date

def PYTESS_extract_digit(sent):
    nmb_list = [s for s in sent.split() if s.isdigit()]
    nmb = "".join(nmb_list)
    nmb_list, nmb, len(nmb)

    if len(nmb) > 8:
        return nmb
    else:
        return None

def PYTESS_name_check(sent):
    sent =  sent.strip()
    upper_res = [char for char in sent if char.isupper()]
    if len(upper_res) > 6 and sent[0].isupper() and sent[1].isupper() and sent[-2].isupper() and sent[-1].isupper():
        #print("*********************************************************")
        return "ok"
    else:
        return "notok"

def Extract_NID_INFO_PYTESS_OCR():
    IMG_DIR = '/content/manual-cropedd'

    name ="Not Found"
    age ="Not Found"
    NID_NUM ="Not Found"
    for img_name in os.listdir(IMG_DIR):
        img_path = IMG_DIR + "/" +img_name
        img = cv2.imread(img_path)

        rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        sent = pytesseract.image_to_string(rgb)
        
        name_status = PYTESS_name_check(sent)
            
        b_date = PYTESS_OCR_month_matching(sent)
        NID_no = PYTESS_extract_digit(sent)
        
        #print("=============> ", sent)
        if name_status == "ok":
            name = sent
            # print(sent)
        if b_date != None:
            age = b_date
            # print(b_date)
        if NID_no != None and len(NID_no) == 10:
            NID_NUM = NID_no
            # print(NID_NUM)
        
    info_dict = dict()
    info_dict["Name"] = name.strip()
    info_dict["Date of Birth"] = age
    info_dict["NID_no"] = NID_NUM

    return info_dict

# EAST Implementation

In [ ]:
def EAST_extract_digit(sent, n_digit):
    sent = sent.strip()
    nmb_list = [s for s in sent.split() if s.isdigit()]
    nmb = "".join(nmb_list)
    nmb_list, nmb, len(nmb)

    if len(nmb) > n_digit:
        return nmb
    else:
        return None


MONTH_NAME = ['Jan', 'Feb', 'Apr' 'Mar', 'May', 'Jun', 'Jul', 'Aug', 'Sept', 'Oct', 'Nov', 'Dec']
def EAST_extract_age(sent):
    sent =  sent.strip()
    word_token = nltk.tokenize.word_tokenize(sent)
    for i, word in enumerate(word_token):
        try:
            if word in MONTH_NAME:
                date_c = word_token[i-1]
                s_d = re.search(r"\d+(\.\d+)?", date_c)
                date = s_d.group(0)

                month = word_token[i]


                year_c = word_token[i+1]
                y_d = re.search(r"\d+(\.\d+)?", year_c)
                year = y_d.group(0)

                if len(date) == 2 and len(year) == 4 and int(date) < 32:
                    b_date = date + " " + month + " " + year
                    # print(b_date)
                    return b_date
        except:
            pass
            
def EAST_extract_name(sent):
    sent =  sent.strip()
    upper_res = [char for char in sent if char.isupper()]
    if len(upper_res) > 6 and sent[0].isupper() and sent[1].isupper() and sent[-2].isupper() and sent[-1].isupper():
        #print("*********************************************************")
        return sent

In [ ]:
def Extract_NID_INFO_EAST_OCR():

    IMG_DIR = '/content/east-cropedd'
    
    name ="Not Found"
    age ="Not Found"
    NID_NUM ="Not Found"
    for img_name in os.listdir(IMG_DIR):
        img_path = IMG_DIR + "/" +img_name
        img = cv2.imread(img_path)

        rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        sent = pytesseract.image_to_string(rgb)

        NID_extract = EAST_extract_digit(sent, 5)
        age_extract = EAST_extract_age(sent)
        name_extact = EAST_extract_name(sent)

        if NID_extract != None and len(NID_extract) == 10:
            NID_NUM = NID_extract
        if age_extract != None:
            age = age_extract
        if name_extact != None:
            name = name_extact
            
    info_dict = dict()
    info_dict["Name"] = name
    info_dict["Date of Birth"] = age
    info_dict["NID_no"] = NID_NUM

    return info_dict




# Making Dataset




In [ ]:
make_dataset('/content/111.png')


[INFO] loading EAST text detector...
[INFO] text detection took 0.590706 seconds


### EASY OCR (OUTPUT)

In [1]:
info_easy_ocr = Extract_NID_INFO_EASY_OCR('/content/NID_cropped.png')
print(info_easy_ocr)

### Tesseract-OCR (OUTPUT)

In [2]:
info_tess_ocr = Extract_NID_INFO_PYTESS_OCR()
print(info_tess_ocr)

# EAST (OUTPUT)

In [3]:
info_east_ocr = Extract_NID_INFO_EAST_OCR()
print(info_east_ocr)